## Reduced Memory Multi-Pass (RMMP) Algorithm
*"Algorithms for Sparse Linear Classifiers in the Massive Data Setting"*, 2008 

**modified shooting algorithm**

- goal: $\beta$ that satisfies $ \max_\beta (\beta ^ T \Psi \beta + \beta ^ T \theta - \gamma ||\beta||_1) $

- "The vector $\Omega$ in the algorithm is defined as $\Omega = 2 \Psi ' \beta + \theta$, where $\Psi '$ is the matrix $\Psi$ with its diagonal entries set to zero. This vector is related to the gradient of the differentiable part of the objective function and consequently can be used for optimality checking."

- "While one can think of numerous stopping criteria for the algorithm, in this paper we stop when successive iterates are sufficiently close to each other (relatively, and with respect to the L2). 
More precisely, we declare convergence whenever
$||\beta_i - \beta_{i-1}||_2 / ||\beta_{i-1}||_2$
is less than some user specified tolerance. Note that $\beta_i$ is the parameter vector at iteration $i$, which is obtained after cycling through and updating all $d$ components once.


![modified shooting pseudocode](images/shooting-pseudocode.png)

**the reduced memory multi-pass (RMMP) algorithm**
![rmmp pseudocode](<images/rmmp-pseudocode.png>)

for $y_i=1$

![ai for yi=1](images/ai-for-yi-1.png)

![bi for yi=1](images/bi-for-yi-1.png)

for $y_i=0$

![ai and bi for yi=0](images/ai-bi-for-yi-0.png)

$c$^ $= \beta_{i-1}^T x_i$

$\Phi$ is the link function, either logistic or probit.

In [22]:
import numpy as np

# ||Bi - Bi-1|| / ||Bi-1|| < tolerance
def has_converged(new_parameters, parameters, tolerance=1e-6):
    return (np.linalg.norm(new_parameters - parameters) / np.linalg.norm(parameters)) < tolerance


def modified_shooting(parameters, d, vector_stats, matrix_stats, tolerance=1e-6):
    print("Modified Shooting")
    # initialize new_parameters
    new_parameters = np.zeros_like(parameters)
    
    # create a new matrix_stats with the diagonals set to 0
    new_matrix_stats = matrix_stats.copy()
    new_matrix_stats[np.diag_indices(d)] = 0
    
    # initialize gradient_vector
    gradient_vector = np.zeros(d)
    
    counter = 0
    
    while True:
        for j in range(d):
            # update new_parameters based on tolerance
            if(abs(gradient_vector[j]  <= tolerance)):
                new_parameters[j] = 0
            elif gradient_vector[j] > tolerance:
                new_parameters[j] = (tolerance - gradient_vector[j]) / (2 * matrix_stats[j, j])
            elif gradient_vector[j] < -tolerance:
                new_parameters[j] = (- tolerance - gradient_vector[j]) / (2 * matrix_stats[j, j])
            
            # update gradient_vector
            gradient_vector = 2 * new_matrix_stats @ new_parameters + vector_stats
            print(f"MS: gradient_vector = {gradient_vector}")
        
        # check for convergence
        if has_converged(new_parameters, parameters, tolerance):
            break
        
        # update parameters
        parameters = new_parameters.copy()
        
        counter += 1
        if counter % 100 == 0:
            break
            print(f"MS: Iteration {counter}: parameters = {parameters}")
    return new_parameters



In [23]:
# logistic link function
def logit_model(x):
    return 1 / (1 + np.exp(-x))

def logit_model_derivative(x):
    return logit_model(x) * (1 - logit_model(x))

def logit_model_double_derivative(x):
    return logit_model_derivative(x) * (1 - 2 * logit_model(x))


# quadratic approximation to term likelihood at parameters
def quad_approximation(xi, yi, parameters, d):
    c = parameters.T @ xi
    print(f"c = {c}")
    
    if yi == 1:
        ai = 0.5 * (logit_model_double_derivative(c) / logit_model(c) 
                    - (logit_model_derivative(c) / logit_model(c)) **2
                    )
        bi = logit_model_derivative(c) / logit_model(c) - c * ai
    # if yi == 0
    else:
        ai = 0.5 * (logit_model_double_derivative(c) / (1 - logit_model(c)) - (logit_model_derivative(c) / (1 - logit_model(c)))**2)
        bi = logit_model_derivative(c) / (1 - logit_model(c)) + c * ai
    
    return ai, bi



def rmmp(X, y, tolerance=1e-6):
    # number of examples and dimension
    t, d = X.shape
    print(f"Number of examples: {t}, Dimension: {d}")
    
    parameters = np.zeros(d) # parameters of the regression model
    active_set = set() # components that are either non-zero and optimal or not optimal
    counter = 1 
    gradient_vector = np.zeros(d)
    
    while True:
        vector_stats = np.zeros(d)
        matrix_stats = np.zeros((d, d)) 
        
        for i in range(t):
            # get ith observation (xi, yi)
            xi = X[i].reshape(-1, 1)
            yi = y[i]
            
            print(f"Observation {i+1}/{t}: xi = {xi.T}, yi = {yi}")
            
            ai, bi = quad_approximation(xi, yi, parameters, d)
            print(f"ai = {ai}, bi = {bi}")
            # update ai and bi
            
            matrix_stats += ai * (xi @ xi.T)
            print(f"matrix_stats = {matrix_stats}")
            vector_stats += bi * xi.reshape(-1)
            print(f"vector_stats = {vector_stats}")
            
            new_matrix_stats = matrix_stats.copy()
            new_matrix_stats[np.diag_indices(d)] = 0
            print(f"new_matrix_stats = {new_matrix_stats}")
            gradient_vector = 2 * new_matrix_stats @ parameters + vector_stats
            print(f"gradient_vector = {gradient_vector}")
            
        new_parameters = modified_shooting(parameters, d, vector_stats, matrix_stats, tolerance)
        print(f"new_parameters = {new_parameters}")
        
        active_set = {j for j in range(d) if gradient_vector[j] >= tolerance}
        print(f"Active set: {active_set}")
        
        if has_converged(new_parameters, parameters):
            break
        
        parameters = new_parameters
        counter += 1
        
        if counter % 100 == 0:
            break
            print(f"Iteration {counter}: parameters = {parameters}")
    
    return parameters, active_set
            


In [24]:
# example usage
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([0, 1, 0, 1])
parameters, active_set = rmmp(X, y)
print("Parameters:", parameters)

Number of examples: 4, Dimension: 2
Observation 1/4: xi = [[1 2]], yi = 0
c = [0.]
ai = [-0.125], bi = [0.5]
matrix_stats = [[-0.125 -0.25 ]
 [-0.25  -0.5  ]]
vector_stats = [0.5 1. ]
new_matrix_stats = [[ 0.   -0.25]
 [-0.25  0.  ]]
gradient_vector = [0.5 1. ]
Observation 2/4: xi = [[3 4]], yi = 1
c = [0.]
ai = [-0.125], bi = [0.5]
matrix_stats = [[-1.25 -1.75]
 [-1.75 -2.5 ]]
vector_stats = [2. 3.]
new_matrix_stats = [[ 0.   -1.75]
 [-1.75  0.  ]]
gradient_vector = [2. 3.]
Observation 3/4: xi = [[5 6]], yi = 0
c = [0.]
ai = [-0.125], bi = [0.5]
matrix_stats = [[-4.375 -5.5  ]
 [-5.5   -7.   ]]
vector_stats = [4.5 6. ]
new_matrix_stats = [[ 0.  -5.5]
 [-5.5  0. ]]
gradient_vector = [4.5 6. ]
Observation 4/4: xi = [[7 8]], yi = 1
c = [0.]
ai = [-0.125], bi = [0.5]
matrix_stats = [[-10.5 -12.5]
 [-12.5 -15. ]]
vector_stats = [ 8. 10.]
new_matrix_stats = [[  0.  -12.5]
 [-12.5   0. ]]
gradient_vector = [ 8. 10.]
Modified Shooting
MS: gradient_vector = [ 8. 10.]
MS: gradient_vector = [-0.

/var/folders/wk/c98ddbnx3pd7fpdy174bcjmr0000gn/T/ipykernel_42711/2148723187.py:5: RuntimeWarning: divide by zero encountered in scalar divide
  return (np.linalg.norm(new_parameters - parameters) / np.linalg.norm(parameters)) < tolerance
/var/folders/wk/c98ddbnx3pd7fpdy174bcjmr0000gn/T/ipykernel_42711/2148723187.py:5: RuntimeWarning: invalid value encountered in scalar divide
  return (np.linalg.norm(new_parameters - parameters) / np.linalg.norm(parameters)) < tolerance


new_matrix_stats = [[ 0.  -5.5]
 [-5.5  0. ]]
gradient_vector = [4.5 6. ]
Observation 4/4: xi = [[7 8]], yi = 1
c = [0.]
ai = [-0.125], bi = [0.5]
matrix_stats = [[-10.5 -12.5]
 [-12.5 -15. ]]
vector_stats = [ 8. 10.]
new_matrix_stats = [[  0.  -12.5]
 [-12.5   0. ]]
gradient_vector = [ 8. 10.]
Modified Shooting
MS: gradient_vector = [ 8. 10.]
MS: gradient_vector = [-0.3333325 10.       ]
MS: gradient_vector = [-0.3333325 10.       ]
MS: gradient_vector = [-0.3333325 10.       ]
new_parameters = [0.        0.3333333]
Active set: {0, 1}
Observation 1/4: xi = [[1 2]], yi = 0
c = [0.6666666]
ai = [-0.32452026], bi = [0.44440953]
matrix_stats = [[-0.32452026 -0.64904052]
 [-0.64904052 -1.29808105]]
vector_stats = [0.44440953 0.88881907]
new_matrix_stats = [[ 0.         -0.64904052]
 [-0.64904052  0.        ]]
gradient_vector = [0.0117159  0.88881907]
Observation 2/4: xi = [[3 4]], yi = 1
c = [1.3333332]
ai = [-0.08254551], bi = [0.31866922]
matrix_stats = [[-1.06742986 -1.63958666]
 [-1.63